## 워크플로우 요약

[선행작업] KoBART로 기사 본문의 [생성 요약문]을 미리 준비합니다.  
[데이터 구축] [정상 제목]+[생성 요약문] / [낚시성 제목] + [생성 요약문]을 페어로 묶어 대조 학습용 데이터셋을 만듭니다.  
[모델 학습] KoSimCSE 모델을 ContrastiveLoss로 파인튜닝합니다.  
[평가] 학습된 모델로 유사도를 계산하고 임계값(Threshold)을 설정합니다.

## 1단계: 선행 작업 - KoBART 요약 모델 (가정)

본 노트북은 1단계가 이미 완료되었다고 가정합니다. 
(보유한 <요약문 생성 데이터>(B)로 KoBART 모델이 이미 파인튜닝되었습니다.)

이 KoBART 모델을 사용하여, <낚시성 뉴스 데이터>(A)의 모든 본문에 대한 [생성된 요약문]을 확보했습니다.

## 2단계: KoSimCSE 학습용 데이터셋 구축 
KoSimCSE 모델을 '지도 학습(Supervised)' 방식의 대조 학습으로 파인튜닝하기 위한 데이터셋을 구성합니다.

Positive Pair (일치, 레이블 1.0)
입력 1 (text1): <낚시성 뉴스 데이터>(A.i)의 [정상 기사 제목]
입력 2 (text2): 1단계에서 생성한 해당 기사의 [생성된 요약문]

Negative Pair (불일치, 레이블 0.0)
입력 1 (text1): <낚시성 뉴스 데이터>(A.ii)의 [일치하지 않는 기사 제목]
입력 2 (text2): 1단계에서 생성한 해당 기사의 [생성된 요약문]

## 3단계: KoSimCSE 모델 구성 및 파인튜닝
sentence-transformers 라이브러리를 사용하여 모델을 구성하고 학습시킵니다.

In [ ]:
#-- 3.1. 필요 라이브러리 설치 (Jupyter/Colab 환경)
# %pip install sentence-transformers torch

#기타 필요 라이브러리
# %pip install datasets
# %pip install accelerate

In [ ]:
# 3.2. 모듈 임포트
import os
from sentence_transformers import SentenceTransformer, InputExample, losses, util
from torch.utils.data import DataLoader
import torch


print("라이브러리 임포트 완료")

라이브러리 임포트 완료


In [3]:
# 3.3. 학습 데이터 정의 (예시)
# 2단계에서 설명한 (text1, text2, label) 형식의 예시 데이터를 사용합니다. 
# 참고: 실제 프로젝트에서는 이 리스트에 수천~수만 건의 데이터를 로드해야 합니다.


# [ (text1: 기사 제목), (text2: 본문 요약문), (label: 1.0=일치, 0.0=불일치) ]
train_data_list = [
    # 예시 1: 김봉진 대표 (사용자 제공 데이터 기반)
    ("40억달러 딜 주인공 김봉진 \"국내서 폼잡다 죽고싶지 않았다\"", 
     "DH가 기업가치를 약 4조 8000억 원으로 평가한 우아한형제들의 김 대표는 국내 스타트업 사상 최대 규모의 M&A 이후 합작법인 우아DH아시아의 책임자가 된다.", 
     1.0), # Positive Pair
    ("‘민족’ 버리고 독일 택했다? 40억달러 김봉진 대표의 충격적 선택", 
     "DH가 기업가치를 약 4조 8000억 원으로 평가한 우아한형제들의 김 대표는 국내 스타트업 사상 최대 규모의 M&A 이후 합작법인 우아DH아시아의 책임자가 된다.", 
     0.0), # Negative Pair
    
    # 예시 2: 부동산 (추가 예시)
    ("서울 아파트 거래량 3개월 연속 감소... 금리 영향 '뚜렷'", 
     "금리 인상과 대출 규제의 영향으로 매수 심리가 위축되면서, 서울 아파트 월간 거래량이 3개월 연속 감소하며 시장 안정화 추세를 보이고 있다.", 
     1.0), # Positive Pair
    ("서울 아파트 '폭락' 시작됐다! 지금 당장 집 팔아야 하는 이유", 
     "금리 인상과 대출 규제의 영향으로 매수 심리가 위축되면서, 서울 아파트 월간 거래량이 3개월 연속 감소하며 시장 안정화 추세를 보이고 있다.", 
     0.0), # Negative Pair
    
    # 예시 3: IT/기술 (추가 예시)
    ("과기부, 차세대 AI 반도체 개발에 5년간 1조원 투입", 
     "정부가 글로벌 AI 시장 경쟁력 확보를 위해, 향후 5년간 1조 원 규모의 예산을 투입하여 시스템 반도체와 AI 기술을 융합한 차세대 반도체 개발을 추진한다.", 
     1.0), # Positive Pair
    ("AI가 인간 일자리 모두 빼앗는다... 정부의 충격적인 '1조원' 계획", 
     "정부가 글로벌 AI 시장 경쟁력 확보를 위해, 향후 5년간 1조 원 규모의 예산을 투입하여 시스템 반도체와 AI 기술을 융합한 차세대 반도체 개발을 추진한다.", 
     0.0)  # Negative Pair
]

print(f"총 {len(train_data_list)}개의 학습 예시 데이터 준비 완료")

총 6개의 학습 예시 데이터 준비 완료


In [4]:
# 3.4.0. 필요한 모듈 추가 임포트
from sentence_transformers import models

# 3.4.1. 사용할 KoSimCSE 모델 이름 (사용자님이 찾으신 모델)
model_name = 'BM-K/KoSimCSE-roberta-multitask'

# 3.4.2. CUDA 사용 가능 여부 체크
device = "cuda" if torch.cuda.is_available() else "cpu"

# 3.4.3. SentenceTransformer 모델 수동 조립
try:
    # 1. '엔진' (기본 Transformer 모델) 로드
    word_embedding_model = models.Transformer(model_name)

    # 2. '풀링 레이어' 정의 (이것이 핵심입니다!)
    # SimCSE는 보통 [CLS] 토큰의 임베딩을 문장 벡터로 사용합니다.
    pooling_model = models.Pooling(
        word_embedding_model.get_word_embedding_dimension(),
        pooling_mode='cls' # 'mean'이 아닌 'cls'로 명시적 지정
    )

    # 3. 두 모듈을 결합하여 최종 SentenceTransformer 모델 생성
    model = SentenceTransformer(modules=[word_embedding_model, pooling_model], device=device)
    
    print(f"모델 '{model_name}' 로드 및 '[CLS] 풀링' 적용 완료.")
    print(f"사용 장치: {device}")
    print("--- (이제 'Creating a new one...' 경고가 뜨지 않아야 합니다) ---")

except Exception as e:
    print(f"모델 로드 중 오류 발생: {e}")
    print("Hugging Face Hub에 연결할 수 있는지, 모델 이름이 올바른지 확인해주세요.")

모델 'BM-K/KoSimCSE-roberta-multitask' 로드 및 '[CLS] 풀링' 적용 완료.
사용 장치: cpu
--- (이제 'Creating a new one...' 경고가 뜨지 않아야 합니다) ---


In [5]:
# 3.5. 학습 데이터셋 형식 변환 (InputExample)
# sentence-transformers 라이브러리의 ContrastiveLoss는 InputExample 형식의 데이터를 사용합니다. 
# 여기서 texts 리스트에는 [text1, text2]가, label에는 0.0 또는 1.0이 들어갑니다.

# (이전 3.3. 셀에서 train_data_list가 정의되었다고 가정합니다.)

train_examples = []
for data in train_data_list:
    text1 = data[0]
    text2 = data[1]
    label = float(data[2])
    train_examples.append(InputExample(texts=[text1, text2], label=label))

print(f"총 {len(train_examples)}개의 InputExample 생성 완료")

# 첫 번째 예시 확인
if train_examples:
    print("\n--- 첫 번째 학습 예시 --- ")
    print(f"Text 1: {train_examples[0].texts[0]}")
    print(f"Text 2: {train_examples[0].texts[1]}")
    print(f"Label: {train_examples[0].label}")

총 6개의 InputExample 생성 완료

--- 첫 번째 학습 예시 --- 
Text 1: 40억달러 딜 주인공 김봉진 "국내서 폼잡다 죽고싶지 않았다"
Text 2: DH가 기업가치를 약 4조 8000억 원으로 평가한 우아한형제들의 김 대표는 국내 스타트업 사상 최대 규모의 M&A 이후 합작법인 우아DH아시아의 책임자가 된다.
Label: 1.0


In [6]:
#-- 3.6. 학습 설정 (DataLoader 및 Contrastive Loss)
# DataLoader: 학습 데이터를 배치(batch) 단위로 모델에 전달합니다.
# ContrastiveLoss: 프로젝트 계획 5.B에서 제안한 대조 학습 손실 함수입니다.
# label=1.0 (Positive) 페어는 임베딩 거리를 가깝게 (0에 가깝게) 만듭니다.
# label=0.0 (Negative) 페어는 임베딩 거리를 margin 값보다 멀게 만듭니다.


# DataLoader 생성
# (실제 데이터는 매우 크므로 배치 사이즈를 16, 32 등으로 설정)
# (예시 데이터는 6개뿐이므로, 여기서는 2로 작게 설정)
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=2)

# Loss 함수 정의: ContrastiveLoss
# (3.4에서 'model' 변수가 성공적으로 로드되었다고 가정)
# margin: Negative Pair가 가져야 할 최소 거리 (기본값 0.5)
train_loss = losses.ContrastiveLoss(model=model, margin=0.5) 

print("DataLoader 및 ContrastiveLoss 정의 완료")

DataLoader 및 ContrastiveLoss 정의 완료


In [15]:
#-- 3.7. 모델 학습 실행
# 이제 model.fit()을 호출하여 파인튜닝을 시작합니다. 
# 학습이 완료되면 지정된 경로에 파인튜닝된 모델 파일들이 저장됩니다.

# (NEW) 1. 로깅 모듈 임포트
import logging
import time  # (NEW) 시간 측정을 위한 모듈 임포트

# (NEW) 2. 로깅 설정 (INFO 레벨)
# model.fit()이 INFO 레벨의 로그를 출력하도록 설정합니다.
# (이 코드는 노트북에서 한 번만 실행하면 됩니다)
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO)

# 학습 파라미터 설정
num_epochs = 3 # 에포크 (실제 대규모 데이터에서는 1~3 사이로 조절)
warmup_steps = 1 # 예시 데이터가 매우 작으므로 1로 설정 (실제로는 100-500)
model_save_path = './my-finetuned-simcse-model' # 학습된 모델이 저장될 경로

print(f"모델 파인튜닝 시작... (Epochs: {num_epochs})")

# (NEW) 학습 시간 측정을 위한 시작 시간 기록
start_time = time.time()

# 모델 파인튜닝
# (3.4의 'model' 변수와 3.6의 'train_loss' 변수를 사용)
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs,
          warmup_steps=warmup_steps,
          output_path=model_save_path,
          show_progress_bar=False)

# (NEW) 종료 시간 기록 및 총 시간 계산
end_time = time.time()
total_time = end_time - start_time

print(f"\n모델 학습 완료! '{model_save_path}' 경로에 저장됨.")

# (NEW) 5. 학습 결과(시간)를 가시성 좋게 출력
print("\n--- 🚀 학습 최종 요약 ---")
print(f"  총 학습 시간  : {total_time:.2f} 초")
print("  (에포크별 Loss는 학습 중 출력된 INFO 로그를 확인하세요)")
print("------------------------")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


모델 파인튜닝 시작... (Epochs: 3)


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

***** Running training *****
  Num examples = 6
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 9
  Number of trainable parameters = 110,618,112


Training completed. Do not forget to share your model on huggingface.co/models =)


2025-11-03 15:26:12 - Save model to ./my-finetuned-simcse-model
Configuration saved in ./my-finetuned-simcse-model/config.json


{'train_runtime': 2.5038, 'train_samples_per_second': 7.189, 'train_steps_per_second': 3.594, 'train_loss': 0.0008616057328051991, 'epoch': 3.0}


Model weights saved in ./my-finetuned-simcse-model/model.safetensors
tokenizer config file saved in ./my-finetuned-simcse-model/tokenizer_config.json
Special tokens file saved in ./my-finetuned-simcse-model/special_tokens_map.json



모델 학습 완료! './my-finetuned-simcse-model' 경로에 저장됨.

--- 🚀 학습 최종 요약 ---
  총 학습 시간  : 3.11 초
  (에포크별 Loss는 학습 중 출력된 INFO 로그를 확인하세요)
------------------------


## 4단계: 모델 평가 및 임계값(Threshold) 설정
학습된 모델(혹은 저장된 모델)을 사용하여 '정상'과 '낚시성'을 얼마나 잘 구분하는지 테스트합니다.

In [16]:
#-- 4.1. 학습된 모델로 유사도 계산 테스트
#중요: 이 단계에서는 학습에 사용하지 않은 별도의 평가(validation/test) 데이터셋을 사용해야 합니다.

# 여기서는 3.7에서 학습이 완료된 model 객체를 그대로 사용하여, 간단한 테스트 케이스의 유사도를 계산해봅니다.

# (선택사항) 만약 스크립트를 재시작했다면, 저장된 모델을 이렇게 로드합니다:
model = SentenceTransformer(model_save_path)
print("저장된 모델 로드 완료")

# --- 테스트 케이스 정의 ---

# 테스트 케이스 1: 의미가 통하는 Pair (Positive)
test_headline_1 = "김봉진, 40억달러 딜로 아시아 11개국 총괄"
test_summary_1 = "우아한형제들 김봉진 대표가 DH와의 M&A로 4.8조 가치를 인정받고, 합작법인 '우아DH아시아'의 책임자가 되었다."

# 테스트 케이스 2: 의미가 어긋나는 Pair (Negative - Clickbait)
test_headline_2 = "김봉진 대표, 회사 매각 후 돌연 잠적... '충격'"
test_summary_2 = "우아한형제들 김봉진 대표가 DH와의 M&A로 4.8조 가치를 인정받고, 합작법인 '우아DH아시아'의 책임자가 되었다."

# --- 각 문장을 임베딩 벡터로 변환 ---
# (3.7에서 학습된 'model' 변수를 사용)
print("테스트 케이스 임베딩 중...")
embedding1 = model.encode(test_headline_1, convert_to_tensor=True)
embedding2 = model.encode(test_summary_1, convert_to_tensor=True)

embedding3 = model.encode(test_headline_2, convert_to_tensor=True)
embedding4 = model.encode(test_summary_2, convert_to_tensor=True)

# --- 코사인 유사도 계산 ---
cos_sim_1 = util.cos_sim(embedding1, embedding2)[0][0]
cos_sim_2 = util.cos_sim(embedding3, embedding4)[0][0]

print("\n--- 유사도 계산 결과 ---")
print(f"[테스트 1: 정상 기사 Pair] 유사도: {cos_sim_1:.4f}")
print(f"[테스트 2: 낚시성 기사 Pair] 유사도: {cos_sim_2:.4f}")

2025-11-03 19:49:35 - Use pytorch device_name: mps
2025-11-03 19:49:35 - Load pretrained SentenceTransformer: ./my-finetuned-simcse-model
loading configuration file ./my-finetuned-simcse-model/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "dtype": "float32",
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertTokenizer",
  "transformers_version": "4.57.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 32000
}

loading weights file ./my-finetuned-simcse-model/model.safetensors
lo

저장된 모델 로드 완료
테스트 케이스 임베딩 중...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


--- 유사도 계산 결과 ---
[테스트 1: 정상 기사 Pair] 유사도: 0.9073
[테스트 2: 낚시성 기사 Pair] 유사도: 0.7710


In [9]:
# 4.2. 최적의 임계값(Threshold) 찾기 (실행 가능한 코드)

from sklearn.metrics import f1_score
import numpy as np

# --- 1. 가상의 평가(Test) 데이터셋 정의 ---
# (실제로는 학습(train) 데이터와 겹치지 않는 별도의 데이터를 사용해야 합니다)

test_pairs = [
    # Positive (일치, 1) 예시 2개
    ("김봉진, 40억달러 딜로 아시아 11개국 총괄", 
     "우아한형제들 김봉진 대표가 DH와의 M&A로 4.8조 가치를 인정받고, 합작법인 '우아DH아시아'의 책임자가 되었다."),
    ("서울 아파트 거래량 3개월 연속 감소... 금리 영향 '뚜렷'",
     "금리 인상과 대출 규제의 영향으로 매수 심리가 위축되면서, 서울 아파트 월간 거래량이 3개월 연속 감소하며 시장 안정화 추세를 보이고 있다."),
    
    # Negative (불일치, 0) 예시 2개
    ("김봉진 대표, 회사 매각 후 돌연 잠적... '충격'",
     "우아한형제들 김봉진 대표가 DH와의 M&A로 4.8조 가치를 인정받고, 합작법인 '우아DH아시아'의 책임자가 되었다."),
    ("서울 아파트 '폭락' 시작됐다! 지금 당장 집 팔아야 하는 이유",
     "금리 인상과 대출 규제의 영향으로 매수 심리가 위축되면서, 서울 아파트 월간 거래량이 3개월 연속 감소하며 시장 안정화 추세를 보이고 있다.")
]

# 위 test_pairs에 1:1로 대응되는 실제 정답 레이블
ground_truth_labels = [
    1,  # Positive
    1,  # Positive
    0,  # Negative
    0   # Negative
]

print("--- 1. 가상 평가 데이터셋 준비 완료 ---")

# --- 2. 모델로 유사도 계산 ---
# (3.4 또는 3.7에서 학습된 'model' 변수를 사용)
# (6번 셀에서 임포트한 'util' (from sentence_transformers import util)을 사용)

print("--- 2. 임베딩 및 코사인 유사도 계산 중... ---")

# 각 문장 쌍을 임베딩
test_embeddings1 = model.encode([pair[0] for pair in test_pairs], convert_to_tensor=True)
test_embeddings2 = model.encode([pair[1] for pair in test_pairs], convert_to_tensor=True)

# 코사인 유사도 계산
similarities = []
for emb1, emb2 in zip(test_embeddings1, test_embeddings2):
    sim = util.cos_sim(emb1, emb2)
    similarities.append(sim.item())

print(f"계산된 유사도 값: {np.round(similarities, 4)}")
print(f"실제 정답 레이블: {ground_truth_labels}")

# --- 3. 최적의 임계값 탐색 (F1-Score 기준) ---

print("--- 3. 최적의 F1-Score 기준 임계값 탐색 중... ---")

best_threshold = 0
best_f1 = 0

# 0.01부터 1.00까지 100개의 임계값 후보를 테스트
for threshold in [i * 0.01 for i in range(1, 101)]:
    # 현재 임계값을 기준으로 예측 (유사도가 임계값보다 크면 1, 아니면 0)
    preds = [1 if s > threshold else 0 for s in similarities]
    
    # F1-Score 계산
    # (pos_label=1: '1'을 Positive 클래스로 간주 / zero_division=0: F1 계산 불가 시 0점 처리)
    f1 = f1_score(ground_truth_labels, preds, pos_label=1, zero_division=0)
    
    # 최고 F1-Score 갱신
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold

print("\n--- 4. 탐색 결과 ---")
print(f"최적의 F1-Score: {best_f1:.4f}")
print(f"이때의 임계값(Threshold): {best_threshold:.2f}")

# (참고) 만약 F1-Score가 0.0으로 나온다면, 
# 데이터가 너무 적거나(4개) 모델이 아직 정상/낚시성을 구분하지 못하는 것입니다.
# 실제 수천 개의 평가 데이터로 테스트해야 의미 있는 임계값을 얻을 수 있습니다.

--- 1. 가상 평가 데이터셋 준비 완료 ---
--- 2. 임베딩 및 코사인 유사도 계산 중... ---
계산된 유사도 값: [0.807  0.9616 0.6899 0.5688]
실제 정답 레이블: [1, 1, 0, 0]
--- 3. 최적의 F1-Score 기준 임계값 탐색 중... ---

--- 4. 탐색 결과 ---
최적의 F1-Score: 1.0000
이때의 임계값(Threshold): 0.69


In [10]:
# 4.1의 테스트 결과에 대해, 4.2에서 찾았다고 '가정한' 임계값 적용

# (예시: 만약 4.2의 과정을 통해 최적의 임계값이 0.65라고 찾았다고 가정)
THRESHOLD = 0.65 

print(f"--- 임계값(Threshold) {THRESHOLD} 적용 시 판정 ---")

# 테스트 1 판정
judgment_1 = '[정상]' if cos_sim_1 > THRESHOLD else '[낚시성]'
print(f"테스트 1 (유사도 {cos_sim_1:.4f}) 판정: {judgment_1}")

# 테스트 2 판정
judgment_2 = '[정상]' if cos_sim_2 > THRESHOLD else '[낚시성]'
print(f"테스트 2 (유사도 {cos_sim_2:.4f}) 판정: {judgment_2}")

--- 임계값(Threshold) 0.65 적용 시 판정 ---
테스트 1 (유사도 0.8070) 판정: [정상]
테스트 2 (유사도 0.6899) 판정: [정상]


## 5단계: 최종 워크플로우 적용 (개념)
이제 완성된 두 모델(KoBART, KoSimCSE)을 사용자의 최종 워크플로우 2.C에 적용합니다.
1. 입력: 새로운 기사 [헤드라인 + 본문]이 입력됩니다.
2. 요약: [본문]을 파인튜닝된 KoBART 모델에 입력하여 [생성된 요약문]을 얻습니다.
3. 임베딩: [헤드라인]과 [생성된 요약문]을 파인튜닝된 KoSimCSE 모델(model.encode)에 넣어 두 개의 임베딩 벡터를 추출합니다.
4. 유사도 계산: 두 벡터의 코사인 유사도(util.cos_sim)를 계산합니다.
5. 판별: 이 유사도 점수를 4단계에서 찾은 **임계값(THRESHOLD)**과 비교하여 '정상'/'낚시성' 여부를 판별합니다.
6. 업데이트: 판별 결과를 바탕으로 해당 언론사의 신뢰도 점수를 업데이트합니다.